# Estimating Binary Logit Models with SciPy Optimize and Generic Likelihood Model

Author: Lachlan Deer, April 2017

In [ ]:
import numpy as np
from scipy import optimize as opt

## Simulate Data

In [2]:
# Model Primitives
np.random.seed(1234567890)

nObs = 1000
beta = np.array([0.5, 0.5] , dtype=float)
income= np.random.uniform(size = nObs) # draws from standard normal
explVar = np.vstack([np.ones(nObs), income]).T

In [3]:
def simulateBinaryLogit(x, beta):
    nObs     = x.shape[0]
    nChoice  = 2;
    
    epsilon = np.random.gumbel(size = [nObs, nChoice])
    beta_augmented = np.vstack([np.zeros(beta.shape), beta])
    utility = x @ beta_augmented.T + epsilon
    return np.argmax(utility, axis=1)

In [4]:
# verify choice data looks like it should
choice = simulateBinaryLogit(explVar, beta)
choice[1:10]

array([1, 0, 1, 1, 0, 1, 0, 0, 1], dtype=int64)

In [5]:
data = np.hstack((choice.reshape(nObs, 1), explVar))
data.shape

(1000, 3)

## Defining Functions for Optimization

In [6]:
def calcLambda(x, beta):
    prob = np.exp(x @ beta)  / (1 + np.exp(x @ beta))
    return prob

In [7]:
def logLike_binaryLogit(beta, y, x):
    choiceProb   = calcLambda(x, beta);
    
    ll_i         = y * np.log(choiceProb) + (1 - y) * np.log(1 - choiceProb);
    logLike      = -(ll_i.sum())
    return logLike

## Running the Optimization

In [8]:
beta0 = np.zeros(2)
out = opt.minimize(logLike_binaryLogit, beta0, args=(data[:,0], data[:,1:]) , method='L-BFGS-B', tol=1e-12)

print('beta hat is:', out.x)

print('value of likelihood at beta:', out.fun)

# how to get SE's?
out

beta hat is: [ 0.40157312  0.5869323 ]
value of likelihood at beta: 633.200201176


      fun: 633.20020117642594
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 30
      nit: 8
   status: 0
  success: True
        x: array([ 0.40157312,  0.5869323 ])

In [16]:
# get standard errors from inverse hessian
np.sqrt(np.diagonal(out.hess_inv.todense()))

array([ 0.15264816,  0.27145198])

## An Intro to Simulated Maximum Likelihood

In [17]:
# Simulated Maximum Likelihood

def logLikeSim_binaryLogit(beta, y, x, nSim):
    np.random.seed(42)
    
    nObs = y.shape[0]
    
    simChoice = np.empty((nObs,nSim))
    simChoice[:] = np.NAN
    
    for iSim in range(0, nSim):
        simChoice[:,iSim] = simulateBinaryLogit(x, beta)
    
    simProb = simChoice.mean(axis=1)
    
    ll_i         = y * np.log(simProb) + (1 - y) * np.log(1 - simProb)
    logLike      = -(ll_i.sum())
    return logLike
    


In [25]:
beta0 = 0.4*np.ones(2)
nSim = 1000

out = opt.minimize(logLikeSim_binaryLogit, beta0, args=(data[:,0], data[:,1:], nSim) , method='L-BFGS-B', \
             options={'gtol': 1e-4, 'eps': 1e-04, 'ftol': 1e-8})

print('beta hat is:', out.x)

print('value of likelihood at beta:', out.fun)

# how to get SE's?
out

beta hat is: [ 0.40980455  0.47007518]
value of likelihood at beta: 634.525545482


      fun: 634.52554548208991
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 133.15737038,   98.31582826])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 204
      nit: 8
   status: 0
  success: True
        x: array([ 0.40980455,  0.47007518])

In [26]:
np.sqrt(np.diagonal(out.hess_inv.todense()))

array([ 0.00105096,  0.00087215])

## Comparing SciPy Results to the Canned Logit Estimator

In [188]:
## Compare to the canned logit model 

import statsmodels.api as sm

logit_mod = sm.Logit(data[:,0], data[:,1:])
logit_res = logit_mod.fit(disp=0)
print('Parameters: ', logit_res.params)

Parameters:  [ 0.40157314  0.58693222]


In [189]:
logit_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      998
Method:                           MLE   Df Model:                            1
Date:                Fri, 14 Apr 2017   Pseudo R-squ.:                0.004845
Time:                        07:50:16   Log-Likelihood:                -633.20
converged:                       True   LL-Null:                       -636.28
                                        LLR p-value:                   0.01303
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.4016      0.135      2.985      0.003         0.138     0.665
x1             0.5869      0.237      2.477      0.013         0.122     1.051
==============================================================================
"""

## Maximum Likelihood Estimations using the GenericLikelihoodModel Class

In [190]:
from statsmodels.base.model import GenericLikelihoodModel


### Standard Likelihood Problem

In [196]:
class MyLogit(GenericLikelihoodModel):
    def loglike(self, params):
        exog = self.exog
        endog = self.endog
        
        choiceProb   = calcLambda(self.exog, params);
        ll_i         = endog * np.log(choiceProb) + (1 - endog) * np.log(1 - choiceProb);
        return ll_i.sum()

In [197]:
sm_logit_manual = MyLogit(data[:,0], data[:,1:]).fit()
print(sm_logit_manual.summary())

Optimization terminated successfully.
         Current function value: 0.633200
         Iterations: 57
         Function evaluations: 112
                               MyLogit Results                                
Dep. Variable:                      y   Log-Likelihood:                -633.20
Model:                        MyLogit   AIC:                             1270.
Method:            Maximum Likelihood   BIC:                             1280.
Date:                Fri, 14 Apr 2017                                         
Time:                        07:58:40                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------

### Example with Simulated ML

In [204]:
class MySimulatedLogit(GenericLikelihoodModel):
    def loglike(self, params):
        exog = self.exog
        endog = self.endog
        
        np.random.seed(42)
        nSim = 1000
    
        nObs = endog.shape[0]
    
        simChoice = np.empty((nObs,nSim))
        simChoice[:] = np.NAN

        for iSim in range(0, nSim):
            simChoice[:,iSim] = simulateBinaryLogit(exog, params)

        simProb = simChoice.mean(axis=1)
        
        ll_i         = endog * np.log(simProb) + (1 - endog) * np.log(1 - simProb);
        return ll_i.sum()

In [218]:
# very sensitive to starting values! (flat likelihood?)

sm_logit_manual = MySimulatedLogit(data[:,0], data[:,1:]).fit(start_params=0.6*np.ones(2))
print(sm_logit_manual.summary())

Optimization terminated successfully.
         Current function value: 0.637048
         Iterations: 24
         Function evaluations: 57
                           MySimulatedLogit Results                           
Dep. Variable:                      y   Log-Likelihood:                -637.05
Model:               MySimulatedLogit   AIC:                             1278.
Method:            Maximum Likelihood   BIC:                             1288.
Date:                Fri, 14 Apr 2017                                         
Time:                        08:17:57                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------